In [4]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

In [5]:
randomState = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=randomState, stratify=Y)

Using device: cuda


In [6]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, randomState=None, oversample=False):
    
    """
    Generates stratified folds with specified normalization.
    
    For list of scalers, see:
    https://scikit-learn.org/stable/api/sklearn.preprocessing.html
    
    For more details on scaling and normalization effects, see:
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#
    
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - Quantile_Transform(output_distribution='uniform')
    
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=randomState)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # Fit the scaler on the training data and transform both train and test sets
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)
        
        if oversample:
            # Oversample the training data if needed (e.g., using SMOTE or similar techniques)
            # This is a placeholder; actual oversampling code should be implemented here
            # X_train_scaled....
            pass
        
        # Convert back to DataFrame to maintain column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
        
        # Ensure 'gender' is still binary (0 or 1)
        if X_train_scaled['Gender'].isin([0, 1]).all():
            kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        else:
            print("Warning: 'gender' column contains unexpected values after scaling.") 
               
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")   
    return kFolds_list

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 

In [7]:
kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, normalisation_method=MinMaxScaler(), n_splits=5, randomState=randomState)

Fold: 1, Train: (4593, 28), Test: (1149, 28)
Fold: 2, Train: (4593, 28), Test: (1149, 28)
Fold: 3, Train: (4594, 28), Test: (1148, 28)
Fold: 4, Train: (4594, 28), Test: (1148, 28)
Fold: 5, Train: (4594, 28), Test: (1148, 28)


In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 


def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [9]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss()
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

In [10]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            # nn.Sigmoid()
   
        )

    def forward(self, x):
        return self.net(x)
    
    def last_layer(self):
        return self.net[-1]

In [11]:
# test_model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=64, dropout=0.5).to(device)
# print(get_feature_count(train_loader))

In [12]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            if scheduler:
                scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
                avg_val_loss = val_loss / len(val_loader)
                print(f"Epoch {epoch}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        else:
            if avg_val_loss*0.95 <= best_val_loss:
                wait += 1
            if wait >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break
    
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)

    return model, accuracy, precision, recall, f1, auc


In [ ]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 1000
    
    
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 28, 128)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    initial_lr = trial.suggest_float("initial_lr", 1e-5, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-3, 1e-1, log=True)
    
    # Loss function hyperparameters
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss"]) #, "FocalLoss", "DiceLoss"])
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=256, device=device)
        # Calculate steps_per_epoch from the current fold's train_loader
        train_loader_len = len(train_loader)
        
        # Instantiate and initialize the model
        model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight).to(device)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr)

        
        # Initialize scheduler
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimiser,
            max_lr=max_lr,
            steps_per_epoch=train_loader_len,
            epochs=epochs,
            anneal_strategy='linear'
        )
        print(f"Fold {fold}:")
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=30, device=device
        )

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5

    return combined_score


In [14]:
import threading
import optuna
from optuna_dashboard import run_server

def start_dashboard():
    run_server(storage)

storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# Start dashboard in a separate thread
dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# After optimization, print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-07 20:08:58,625] A new study created in memory with name: Basic


Using device: cuda


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Fold 1:
Epoch: 1, training loss: 1.1141
Epoch 1, Val Loss: 1.0847
Epoch: 2, training loss: 1.0941
Epoch 2, Val Loss: 1.0693
Epoch: 3, training loss: 1.0763
Epoch 3, Val Loss: 1.0569
Epoch: 4, training loss: 1.0604
Epoch 4, Val Loss: 1.0445
Epoch: 5, training loss: 1.0502
Epoch 5, Val Loss: 1.0315
Epoch: 6, training loss: 1.0476
Epoch 6, Val Loss: 1.0194
Epoch: 7, training loss: 1.0253
Epoch 7, Val Loss: 1.0058
Epoch: 8, training loss: 1.0210
Epoch 8, Val Loss: 0.9925
Epoch: 9, training loss: 1.0114
Epoch 9, Val Loss: 0.9804
Epoch: 10, training loss: 1.0010
Epoch 10, Val Loss: 0.9698
Epoch: 11, training loss: 0.9913
Epoch 11, Val Loss: 0.9572
Epoch: 12, training loss: 0.9781
Epoch 12, Val Loss: 0.9481
Epoch: 13, training loss: 0.9670
Epoch 13, Val Loss: 0.9407
Epoch: 14, training loss: 0.9677
Epoch 14, Val Loss: 0.9340
Epoch: 15, training loss: 0.9543
Epoch 15, Val Loss: 0.9282
Epoch: 16, training loss: 0.9514
Epoch 16, Val Loss: 0.9243
Epoch: 17, training loss: 0.9623
Epoch 17, Val Los

[I 2025-04-07 20:09:19,411] Trial 0 finished with value: 0.23978233935069304 and parameters: {'hidden_dim': 73, 'dropout': 0.4068561601830548, 'initial_lr': 5.0038873401179773e-05, 'max_lr': 0.017289824638793267, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 7}. Best is trial 0 with value: 0.23978233935069304.


Epoch: 54, training loss: 0.8809
Epoch 54, Val Loss: 0.9732
Epoch: 55, training loss: 0.8941
Epoch 55, Val Loss: 0.9729
Epoch: 56, training loss: 0.8772
Epoch 56, Val Loss: 0.9810
Early stopping triggered at epoch 56
Using device: cuda
Fold 1:
Epoch: 1, training loss: 1.1870
Epoch 1, Val Loss: 1.1523
Epoch: 2, training loss: 1.1479
Epoch 2, Val Loss: 1.1161
Epoch: 3, training loss: 1.1148
Epoch 3, Val Loss: 1.0836
Epoch: 4, training loss: 1.0876
Epoch 4, Val Loss: 1.0541
Epoch: 5, training loss: 1.0686
Epoch 5, Val Loss: 1.0281
Epoch: 6, training loss: 1.0497
Epoch 6, Val Loss: 1.0124
Epoch: 7, training loss: 1.0324
Epoch 7, Val Loss: 1.0014
Epoch: 8, training loss: 1.0283
Epoch 8, Val Loss: 0.9907
Epoch: 9, training loss: 1.0265
Epoch 9, Val Loss: 0.9879
Epoch: 10, training loss: 1.0096
Epoch 10, Val Loss: 0.9871
Epoch: 11, training loss: 1.0030
Epoch 11, Val Loss: 0.9856
Epoch: 12, training loss: 0.9991
Epoch 12, Val Loss: 0.9837
Epoch: 13, training loss: 1.0007
Epoch 13, Val Loss: 0

[W 2025-04-07 20:10:32,213] Trial 1 failed with parameters: {'hidden_dim': 83, 'dropout': 0.3787829298279479, 'initial_lr': 0.00013805965529992457, 'max_lr': 0.03709577243188602, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_17152\2883108387.py", line 56, in maximise_combined_score
    model, accuracy, precision, recall, f1, auc = train_and_evaluate(
                                                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_17152\439299055.py", line 15, in train_and_evaluate
    for batch, (inputs, labels) in enumerate(train_loader,start=1):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\torch\utils\data\dataloader.py", lin

KeyboardInterrupt: 